<a href="https://colab.research.google.com/github/scskalicky/LING-226-vuw/blob/main/15_regular_expressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Regular Expressions

Regular expressions are a method for finding patterns in text. They are not specific to Python, and can be used in a variety of programming languages as well as applications that allow for regular expressions. You might see them called `regex` for short, but also referred to as `grep` and other terms, which indicate the primary functions used for regular expressions.

Learning regular expressions is an incredibly important part of programming in general and NLP specifically. Regular expressions allow you to specify extremely precise search patterns, and perform operations on the results of those patterns. 

You can use regular expressions to search for certain filenames in a directory or certain morphemes in a word. 

In this notebook, we'll learn the basics of regex which will help us understand how NLTK has employed regex for linguistic analysis. 

To use regular expressions in Python, we need to `import re`

In [ ]:
# import re (regular expressions module)
import re

## creating regex patterns and `re.search()`

All regex searches require defining a search pattern — you need to tell Python what it is you're looking for!

Search patterns are entered as strings, but can include special characters to allow for variability and option that otherwise cannot be included in just a string. You can search for:

- Actual characters/strings (e.g., searching for specific words or parts of words). 
  - These characters mean you want Python to search for the literal version of what you typed (e.g., if you typed "hello!" you would want to search for that exact word, including the exclamation mark)
  - in other words, this asks for Python to search for that specific string

- meta characters which represent something beyond their literal version. We have already seen some of these characters, such as `\n` for newlines and `\t` for tab.

- Quantifiers, which allow you to constraint the number of times something repeats (e.g., asking for any number of characters followed by a number). 





### Searching through NZ Birds of the Year

Let's start with a small-scale example. Let's pretend we want to search for various linguistic properties of the names of all [NZ Bird of the Year Winners](https://en.wikipedia.org/wiki/Bird_of_the_Year):

```
2022	Pīwauwau
2021	Pekapeka-tou-roa
2020	Kākāpō
2019	Hoiho
2018	Kererū
2017	Kea
2016	Kōkako
2015	Kuaka
2014	Tara iti
2013	Mōhua
2012	Kārearea
2011	Pūkeko
2010	Kākāriki
2009	Kiwi
2008	Kākāpō
2007	Riroriro
2006	Pīwakawaka
2005	Tūī
```

Let's first get those winners into a list!

In [ ]:
# define list of BoTY winners:
birbs = ['Pīwauwau', 'Pekapeka-tou-roa', 'Kākāpō', 'Hoiho', 'Kererū', 'Kea', 'Kōkako', 'Kuaka', 'Tara-iti',
         'Mōhua', 'Kārearea', 'Pūkeko' ,'Kākāriki', 'Kiwi', 'Kākāpō', 'Riroriro', 'Pīwakawaka', 'Tūī']

Okay, now we can start using `re.search()` as a means to find if any of our bird names contain specific characters or patterns.

`re.search()` is a function which results in two outcomes:

1. If the pattern exists in the string being searched, `re.search()` will return a `match` object which contains details of the match
2. If the pattern does not exist in the string being searched, `re.search()` will return nothing

With this knowledge, we can use `re.search()` as a *conditional* test to return strings which do or do not have specific patterns of interest. More specifically, we can specify `if re.search():` which would only return `True` if there was a match in the string. 

The syntax of `re.search()` is:

```
re.search(pattern = '', string = '')
```

So we need to first supply the pattern we are looking for, and then supply the string we are seaching. 


Knowing this,  let's search for capital "K" in our list of birds and print any bird that includes a capital K:

In [ ]:
# let's save our search pattern to a variable:
pattern = 'K'


# create a for loop through our list of birds
for birb in birbs:
  # if there is a resulting match object
  if re.search(pattern , birb):
  # print the bird
    print(f'{birb} has a {pattern}!')
  else:
    print(f'{birb} does not have a {pattern}!')

One thing you should notice about those results is that the search is quite literal - birds with only lowercased "k" such as Pūkeko did not meet the condition of the test - which makes sense. We could repeat the same search as above but with a lowercased 'k':

In [ ]:
# same as previous but w/ lowercase k
pattern = 'k'

# create a for loop through our list of birds
for birb in birbs:
  # if there is a resulting match object
  if re.search(pattern , birb):
  # print the bird
    print(f'{birb} has a {pattern}!')
  else:
    print(f'{birb} does not have a {pattern}!')

We run into the converse situation here — birds with only uppcase "K" such as 'Kiwi' were not matched. 

Usually, we want to add a bit more flexibilty into our searches so that we can find strings that meet a variety of criteria. In this case, how could we create a pattern which would allow for birds including both upper and lowercased 'k' to be returned?

Clearly, one option would be to convert the strings to lowercase before doing any searches! But, keep in mind that we are dealing with proper nouns here (i.e., names of things), and in English these are signalled in writing through capitalization of the initial letter. So in this case, applying a lowercase to everything might not be appropriate. 


So, let's use this opportunity to see how to allow for options in our search patterns. We can use square brackets to tell `re.search()` that we are looking for *any* candidate within the square brackets:

```
'[Kk]' = any of 'K' or 'k'
```

In this way, the square brackets are effectively a shorthand for using `or`. Let's demonstrate this, while also exploring how to use the `match` object to show us which part of the pattern was actually a hit. We can do so by using the `.group()` method from a resulting regex match. 



In [ ]:
# define our search pattern
pattern = '[Kk]'
# create a for loop through our list of birds
for birb in birbs:
  # save the match to a variable
  match = re.search(pattern, birb)
  # if there is a match, print the bird and the results of the match
  if match:
  # print the bird
    print(f'{birb} has a {match.group()}!') 
  else:
    print(f'pattern {pattern} not found for {birb}!')

Looking at the output, we can see that the search was lazy and stopped once it found the first match. Crucially, the search begins from the start of the string, so all birds that start with `K` will be matched for uppercase K, but we still do not know if they contain a lower-cased `k`. In order to answer this relatively silly question of how many "k"s are in the bird names, we would have to take a different approach. 

But let's move on and instead refine a method to search specific parts of a string. Let's stick with our birds for just a little bit longer. First, let's revise our search pattern so that it searches for a `k` *within* the bird name, but *not* the first or last letter of the name. But, in order to do so, we need to understand the use of certain meta characters which designate the start/end of a string, as well as meta characters which stand for any character. 

The full stop `.` is a meta character which stands for *almost* anything (it excludes newlines, for example). You can think of the full stop as a wild card — it can represent anything you want. 

So, a regex pattern comprised of three full stops `'...'` would represent `three of any character in a row`. Such a pattern would match the first combination of three characters in a string, regardless of whether the string is longer than three characters. However, the pattern would *not* match a string of two characters.



### Wildcard: the full stop `.` in regex

In [ ]:
three_dots = '...'

In [ ]:
# Returns a match object
re.search(three_dots, 'dog')

In [ ]:
# also returns a match object (the first 3 characters of the string):
re.search(three_dots, 'dogs and cats')

In [ ]:
# Returns nothing because there is no sequence of three characters in this string
re.search(three_dots,'ab')

In [ ]:
# Returns nothing because `.` cannot stand for newlines. 
re.search(three_dots, 'ab\n')

### String anchors: `^` and `$`

Remember the challenges associated with defining a word to make tokens? Regex has special characters which sort of help with this, in that they represent the *start* and the *end* of a string (which, of course, is not always the same as a word!).

These characters are also called `anchors`:

```
^ - start of a string
$ - end of a string
```

So we can use these strings to better constrain our search patterns. If instead of finding any three-character sequence, we wanted to find any three-character *word*, we could revise our three dots pattern as such:

In [ ]:
# regex search pattern which looks for any three characters between a start/stop of a string
three_character_word = '^...$'

In [ ]:
# it will find dog...
re.search(three_character_word, 'dog')

In [ ]:
# and cat...
re.search(three_character_word, 'cat')

In [ ]:
# but not bird. (because there are four characters between start/stop)
re.search(three_character_word, 'bird')

### Quantifiers

So regex patterns allowed for us to scope the nature of our search to words of certain lengths, as well as to avoid characters that are at the start and end of a word. 


Let's extend this knowledge to return to our birds and find if we can define a search pattern which locates lowercase 'k' in bird names, even if they start with an uppercase 'k'. We need to learn *one more thing* though, and that's how to attach quantifier flags to characters. Quantifers allow us to specify how many times the *preceding* character can occur in the search. One of the most useful quantifiers is `*` which stands for `zero or more`, a rather liberal constraint on a character. 

So, if we wanted to search for "zero to infinite instances of any character", we could type `.*`, for example:

In [ ]:
# .* basically says "find me anything"
re.search('.*', 'these pretzels are making me thirsty!')

In [ ]:
# even nonsense
re.search('.*', 'asdfasdlfkjasdl;k#fjasd;lk@fj)))')

### Finally back to the birds!

So, using `.*` is quite dangerous unless we constrain the search a bit more. Let's apply this to our birds problem and define a search pattern which finds a `k` occuring inside the name of our birds. Here is the pattern:

```
`'^..*k.*.$'`

```

This pattern says search for:

`^` (start of a string), `.` (followed by one wild card - the first letter of the name), `.*` (followed by zero or more wildcards), `k` followed by one "k", `.*` (followed by zero or more wildcards), `.` (followed by one wild card - the last letter of the name), `$` (followed by the end of the string). 

This pattern lets us find a lowercase k within a string of any length. 

In [ ]:
mid_k = '^..*k.*.$'

In [ ]:
pattern = mid_k
# create a for loop through our list of birds
for birb in birbs:
  # save the match to a variable
  match = re.search(pattern, birb)
  # if there is a match, print the bird and the results of the match
  if match:
  # print the bird
    print(f'{birb} has a lower-cased k!') 
  else:
    print(f'{birb} does not have a lower case k!')

### Regular expressions are challenging

Learning regular expressions is challenging, and `re.search()` and `re.match()` are probably the more annoying ones to work with. 


The rest of this notebook covers some of the material from the NLTK book, and ideally this digression helps to digest the NLTK material. A really useful website for Regex is [Regex101](https://regex101.com/) (make sure you select Python), and there are a number of other resources out there to help you learn regex, such as [Geeks for Geeks](https://www.geeksforgeeks.org/python-re-search-vs-re-match/?ref=gcse).

The next section will become a bit repetitive, but I think it is important for something like regex to be repeated. 



# Regex and NLTK


Now that we have loaded in the necessary resources, we can use some built-in data from NLTK to provide us with a list of words. This resource is literally just a list of English words. In the cell below, we use a `list comprehension` to return a lowercase version of every word in the English list of words and save it to a variable named `wordlist`

There are a number of other functions from `re`, and NLTK has also modified or uses regex in some of their custom functions. We will load in NLTK and download some resources.


In [ ]:
import nltk
nltk.download('book')

NLTK has wordlists built in. In the next cell, a `list comprehension` is used to return a lower-cased version of each word from the English set of words, which as saved to a variable named `wordlist`. 

In [ ]:
# following NLTK, get a list of lower case words to use as examples
wordlist = [w for w in nltk.corpus.words.words('en') if w.islower()]

In [ ]:
# there are a lot of words in here!
len(wordlist)

In [ ]:
# sample a random word
wordlist[1337]

## `re.search()` to find `ed` words.

The first regex function NLTK shows you is `re.search()`, which we have already explored above. In the example below, the pattern looks for English words which end in `ed`. Remember, the `$` represents the end of a string. Because we are working with a word list, we know each string represents a single word.

Curious, do you know why `ed$` might be an interesting pattern to search for? That's because most verbs in English which end in "ed" are marking past-tense, and "ed" is therefore a productive suffix performing a mophological task. 

In [ ]:
# this example loops through each word in wordlist and checks if it ends in ed
[w for w in wordlist if re.search('ed$', w)]

As we already know, regular expressions allow you to search for literal sequences of letters, such a `ed` in the above example. Here is another example:

In [ ]:
# create a smaller example to search through
vuw = ["Victoria", "University", "of", "Wellington"]

In [ ]:
# let's search for the word "Victoria" - 
# Remember that this is a very weak use of regular expressions because we are only searching for one specific string. 
for word in vuw:
  if re.search('Victoria', word):
    print(word)

In [ ]:
# The search can be for substrings as well, or even single letters.

# here we will return anything that contains the pattern 'o'
for word in vuw:
  if re.search('o', word):
    print(word)

In [ ]:
# Remember that regex is case sensitive - why doesn't this pattern return a word?
for word in vuw:
  if re.search('wellington', word):
    print(word)
  else:
    print('found nothing')

## End of words

The NLTK example uses the metacharacter `$` so that only the ends of words are matched - otherwise any word containing `ed` as part of the string *anywhere in the string* would be matched. 

In [ ]:
# what does the $ mean?
welly = ['Wellington', 'wellington', 'Wellington!']

# because the third 'wellington' has a '!' at the end, it will not be returned
for word in welly:
  if re.search('ton$', word):
    print(word)

## Wildcard again!

Remember, the `'.'` (full stop) character means *match anything, one time* — it represents any possibility. In the code below, I save the results of each `re.search()` to a variable, then print that variable using the `.group()` method, which returns the actual string result. As I loop through increasing numbers of subsequent full stops, the search results grow. 

In [ ]:
# what does the '.' mean?
s = 'soda'

more_dots = ['.','..', '...', '....']

for dot in more_dots:
  # each time this loops, it uses increasing number of dots
  match = re.search(dot, s)
  print(match.group())

## Start of words


The `^` character is the opposite of the `$` in that it indicates the start of a string. So you can look for things at the start/end of words using these patterns, and also specify the entire lenght of a word you are looking for

In [ ]:
# the NLTK example shows how to use wildcards to make slots for words

# look for all words which start with an 's', end with an 's', and have two characters in between
[w for w in wordlist if re.search('^s..a$', w)]

## **Groups and options**

Did you ever text on a mobile phone before smart phones? Each number on the dial pad represented three possible letters, so you would choose the number for the letters, then press 1, 2, or 3 to choose the letter. For example, the letters. (abc) were on the 1 key, so to type 'a', you would push 1, 1, to type 'b', you would push, 1, 2, and so on. It was quite cumbersome. 

There is a scene in the movie *The Departed* where Matt Damon sends a text using this method while holding his phone in his pocket without looking at it - seems difficult!

The NLTK book uses the example of texting this way to extend our knowledge of regular expressions and the **optional sequences** indicated by square brackets `[]`. These provide a list of options to the search pattern, where the pattern looks for *any* but not all of the things inside the brackets. 

Here's another example.

In [ ]:
# using brackets to find patterns
us_spies = ['fbi', 'cia', 'nsa']

for spy in us_spies:
  if re.search("[fcn][bis][ia]", spy):
    print(spy)

In [ ]:
# the NLTK example shows you how to find "textonyms"
# why are only four words found? Can you find other words using other patterns?
[w for w in wordlist if re.search('^[ghi][mno][jlk][def]$', w)]

In [ ]:
# the '+' sign matches "one or more" - so the results range from one letter to 9
[w for w in wordlist if re.search('^[ghijklmno]+$', w)]

In [ ]:
# the chat corpus is funny to see repeated letters
chat_words = sorted(set(w for w in nltk.corpus.nps_chat.words()))

[w for w in chat_words if re.search('^[ha]+$', w)]

In [ ]:
# how many variations of LOL are there? 
[w for w in chat_words if re.search('^[lL][oO][lL]$', w)]

## Finding regular and irregular past tense.

Regular expressions become more complex as you add additional options to them. It is quite rare that you will want to search for just a word string, usually you will want to search for a variety of candidates that meet a certain condition - what if we wanted to write a regex that found all regular and irregular past tense in English?

The following pattern looks for words that end in either `ed` or `en`. How else can we write the pattern? I've put three versions below that all use the meta characters in different ways. Of course, this also matches word with are *not* past tense but yet still end in these patterns, and there are also even more irregular past tense that are not captured (such as `sank`). Hopefully this gives you some understanding into how powerful regex patterns can be, but also that they will require some trial and error. 



In [ ]:
# using brackets
[w for w in wordlist if re.search('e[dn]$', w)]

In [ ]:
# using |
[w for w in wordlist if re.search('ed$|en$', w)]

In [ ]:
# using () and |
[w for w in wordlist if re.search('(ed|en)$', w)]

As I said above, it is sometimes easy to get frustrated with regular expressions. The NLTK book uses a sort of "try it and figure it out" approach, which can work for some. You might also find resources such as [RegexOne](https://regexone.com/) to be useful in order to fully master regular expressions. 



# ` re.findall()` - Extracting and Manipulating patterns

Besides searching, you might also need to extract some information from words and perform some operation on that information. NLTK introduces the `re.findall()` function. When we used the `re.search()` function, we returned a match object that required some additional processing to get the actual string, so `re.search()` is useful for testing whether strings meet certain conditions, whereas `re.findall()` helps return the actual results. 

In the next example, the pattern searches for any of the vowels inside the square brackets, and using `.findall()` means that every single part of the string which meets that pattern will be returned.

You can see that different words have a different number of vowels, and the entire set of matches is returned as a list. 

In [ ]:
# find all the vowels in VUW
vuw = ['victoria', 'university', 'of', 'wellington']


for word in vuw:
  # findall does what it sounds like it does. 
  print(re.findall('[aeiou]', word))

This means that we can start to *count* the occurance of patterns, using functions such as `nltk.FreqDist()`. 

In the example below, the quantifer `{3}` is used. The squiggle brackets allow you to specify the a precise number of occurences for your pattern, either as a fixed number `{3}` = three times, or as a range: `{3,4}` = three to four times. 



In [ ]:
# let's find all stretches of three letters which only include a certain subset of letters
vuw_triplets = nltk.FreqDist(vowel for word in vuw for vowel in re.findall(r'[vuinc]{3}', word))

vuw_triplets

In [ ]:
# what happens when we reverse sort the list? 
sorted(vuw_triplets, reverse = True)

The **Your Turn** from NLTK is a good challenge. It prompts you to consider yet another meta character, the `\d` which stands for "digit" (i.e., numbers). 

> *Your Turn: In the W3C Date Time Format, dates are represented like this: 2009-12-31. Replace the ? in the following Python code with a regular expression, in order to convert the string '2009-12-31' to a list of integers [2009, 12, 31]:*

> `[int(n) for n in re.findall(?, '2009-12-31')]`


The meta character `\d` matches digits, which is one way to go about this. Can you figure out other ways? 

In [ ]:
[int(n) for n in re.findall('\d+', '2009-12-31')]

## Using `.findall()` to delete all of the vowels in a word

As you get more familiar with regular expressions, or search for more help online, you'll see that a lot of the time, the regular expression patterns are saved to a variable and then used in regex functions. That's what the NLTK authors do below, when showing how to remove all of the vowels from a word. 

You'll also see that an "r" is placed at the start of the regex pattern. This means the string is treated as a "raw" string and has implications for "escaping" characters that we will look at later on. 

Return to the NLTK example of how to remove all vowels from a word. The frustrating part of this example is the dual purpose of the `^`. When the `^` is *outside* of square brackets, it stands for the start of a string, as we've already explored. When the `^` is *inside* square brackets, it is actually *negating* the pattern, so while `'[AE]'` means either anything that is "A" or "E", `'[^AE]'` means anything that is not either "A" or "E".

In [ ]:
# this one is tricky to wrap your head around 
# because the carrot inside the third set of vowels is negating the match...
# so it's anything BUT a vowel - might be hard to notice at first

# save the regex pattern
regexp = r'^[AEIOUaeiou]+|[AEIOUaeiou]+$|[^AEIOUaeiou]'

def compress(word):
  pieces = re.findall(regexp, word)

  # do you remember how ''.join works?
  return ''.join(pieces)

In [ ]:
compress('victoria university of wellington')

In [ ]:
compress('regular expressions are hard!')

## Distribution of sounds in a word
The NLTK authors load in words from the [Rotokas language](https://en.wikipedia.org/wiki/Rotokas_language) and search for CV pairs which stand for consonant-vowel pairs. Consonants and vowels tend to pattern together in systematic ways in languages. 


A conditional frequency distribution is  used to show this distribution. Compare the `s` and `t` rows - `s` can be followed by `i` but is only rarely followed by any other vowls, whereas `t` can be followed be `a` and `o` but not `i`. 




In [ ]:
# using cfd to find minimal pairs. 
rotokas_words = nltk.corpus.toolbox.words('rotokas.dic')
cvs = [cv for w in rotokas_words for cv in re.findall(r'[ptksvr][aeiou]', w)]
cfd = nltk.ConditionalFreqDist(cvs)
cfd.tabulate()

The next section shows you more about using brackets to scope your regex properly, and also how to built a simple stemmer (i.e., a program which separates the roots of a word from the suffix.) The book asks you whether you can spot the problems with the stemmer as presented, the main problem being that many false positives will be returned through using a strict rule-based approach. Why is this? Because English morphology has many rules but also many exceptions to those rules.

NLTK has a neat function, `re_show` which will show you the matches a regex search will make (on a string input)

In [ ]:
# nltk re_show will print the regex matches for your pattern
nltk.re_show(r'vuw', 'vuw')

In [ ]:
nltk.re_show('\d', '2009-12-31')

In [ ]:
us_spies = 'fbiciansa'

nltk.re_show("[fcn][bis][ia]", us_spies)

# NLTK pattern searches

One of the benefits of using NLTK is that the authors have provided their own versions of functions and processes for text analysis. To that end, they introduce their own version of `findall` which uses slightly different syntax to search. This is in many ways is an easier and perhaps more readable way to use regular expressions. Each set of crocodile brackets `<>` defines the boundaries of a match — which could be a word, a single letter, or whatever you please, based on using meta characters. Importantly, the NLTK `.findall()` is different from the regex `.findall()`. 

The NLTK version is a method that is used on an `nltk.Text` object which has been tokenized. The regex version is used on strings. Hopefully these examples make the difference clear. 

In [ ]:
# the re.findall is what we have been using above - it requires a string as input
re.findall(r"these", 'these pretzels')

In [ ]:
# you can use a list, but you must loop the list
for word in ['these', 'pretzels']:
  print(re.findall(r'these', word))

In [ ]:
# can we use the NLTK .findall on a list? no - because it is not an nltk.Text object 
# (please note the error here tells us exactly what the problem is.)
['these', 'pretzels'].findall(r"<these>")

In [ ]:
# so we must remember to convert the strings into an nltk.Text object in order to use the NLTK specific method for searching tokens. 
nltk.Text(['these', 'pretzels']).findall(r"<these>")

Ok, now that we've covered the differences between the two versions of `findall`, let's follow NLTK's lead and do some more interesting stuff using their regex patterns. We need to combine the basic regex syntax (using meta characters and strings) with the angle bracket notation for tokens used by the NLTK version. 

In [ ]:
# Let's convert the brown corpus to an nltk.Text object
from nltk.corpus import brown
brown_words = nltk.Text(brown.words())

In [ ]:
# what kind of stuff will we find with this pattern?
# this pattern means as WORD as WORD
brown_words.findall(r"<as> <\w*> <as> <\w*>")

In [ ]:
# and this one?
brown_words.findall(r"<a> <\w*> <as> <\w*>")

Why would one want to use the NLTK `findall` version instead of regular regex? 

For one thing, it's probably a bit easier to read and understand, but it also lets us think about larger patterns. You'll also see later on that this format can be extended to other types of searches. 

However, the requirement that the text be an `nltk.Text` object is something you will need to remember. 